<a href="https://colab.research.google.com/github/MiguelEuripedes/embedded_AI/blob/main/Projects/first_image_classifier/knn_classifier/Train_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your First Image Classifier: Using k-NN to Classify Images

## Train

The purpose of this dataset is to correctly classify an image as containing a dog, cat, or panda. Containing only 3,000 images, the Animals dataset is meant to be another introductory dataset that we can quickly train a KNN model and obtain initial results (no so good accuracy) that has potential to be used as a baseline.

Let's take the following steps:

1. Encoding target variable
2. Training the KNN model
3. Export the model


### Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:

* **pip install wandb** – Install the W&B library
* **import wandb** – Import the wandb library
* **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 162 kB 71.8 MB/s 
     |████████████████████████████████| 182 kB 49.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 162 kB 71.9 MB/s 
     |████████████████████████████████| 158 kB 74.0 MB/s 
     |████████████████████████████████| 157 kB 60.7 MB/s 
     |████████████████████████████████| 157 kB 55.7 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 66.7 MB/s 
     |████████████████████████████████| 157 kB 60.9 MB/s 
     |████████████████████████████████| 157 kB 55.3 MB/s 
     |████████████████████████████████| 157 kB 56.6 MB/s 
     |████████████████████████████████| 156 kB 74.3 MB/s 


In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#### Import Packeges

Import the necessary packages

In [ ]:
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

Configure logging reference for a logging object

In [ ]:
logger = logging.getLogger()

Set level of logging

In [ ]:
logger.setLevel(logging.INFO)

Create handlers

In [ ]:
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

Add handler to the logger

In [ ]:
logger.handlers[0] = c_handler

### Step 02: Trainning

Since we are using Jupyter Notebooks we can replace our argument parsing code with *hard coded* arguments and values

In [ ]:
args = {
  "project_name": "first_image_classifier",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  "neighbors": 1,
  "jobs": -1,
  "encoder": "target_encoder",
  "inference_model": "model"
}

Open the W&B project created in the Fetch step

In [ ]:
run = wandb.init(entity="euripedes",project=args["project_name"], job_type="Train")

wandb: Currently logged in as: euripedes. Use `wandb login --relogin` to force relogin


In [ ]:
logger.info("Downloading the training data")
train_x_artifact = run.use_artifact(args["train_feature_artifact"])
train_x_path = train_x_artifact.file()
train_y_artifact = run.use_artifact(args["train_target_artifact"])
train_y_path = train_y_artifact.file()

13-10-2022 01:44:17 Downloading the training data


Unpacking the artifacts

In [ ]:
train_x = joblib.load(train_x_path)
train_y = joblib.load(train_y_path)

 Encode the labels as integers

In [ ]:
le = LabelEncoder()

In [ ]:
train_y = le.fit_transform(train_y)

Train a k-NN classifier on the raw pixel intensities

In [ ]:
logger.info("[INFO] training k-NN classifier...")

13-10-2022 01:44:34 [INFO] training k-NN classifier...


In [ ]:
model = KNeighborsClassifier(n_neighbors=args["neighbors"],n_jobs=args["jobs"])
model.fit(train_x, train_y)

KNeighborsClassifier(n_jobs=-1, n_neighbors=1)

In [ ]:
logger.info("Dumping the model and encoder artifacts to the disk")

13-10-2022 01:44:38 Dumping the model and encoder artifacts to the disk


Save the artifacts using joblib

In [ ]:
joblib.dump(le, args["encoder"])
joblib.dump(model, args["inference_model"])

['model']

Encoder artifact

In [ ]:
artifact = wandb.Artifact(args["encoder"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the target encoder"
                          )

logger.info("Logging the target encoder artifact")
artifact.add_file(args["encoder"])
run.log_artifact(artifact)

13-10-2022 01:44:50 Logging the target encoder artifact


Inference model artifact

In [ ]:
artifact = wandb.Artifact(args["inference_model"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the inference model"
                          )

logger.info("Logging the inference model artifact")
artifact.add_file(args["inference_model"])
run.log_artifact(artifact)

13-10-2022 01:44:52 Logging the inference model artifact


In [ ]:
run.finish()